In [1]:
import pandas as pd
import numpy as np 
import requests 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup

import geocoder

In [2]:
# Create the dataframe

df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [3]:
# Get the wikipedia html using Beautiful Soup

html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_data,"html.parser")

In [4]:
# Search for the tables in the HTML data

tables = soup.find_all('table')

In [5]:
# Search for the specific table (easier to parse it and get only the desired table)
for index,table in enumerate(tables):
    if ("Postal Code" in str(table)):
        table_index = index
print(table_index)
tables[table_index].prettify()

0


'<table class="wikitable sortable">\n <tbody>\n  <tr>\n   <th>\n    Postal Code\n   </th>\n   <th>\n    Borough\n   </th>\n   <th>\n    Neighbourhood\n   </th>\n  </tr>\n  <tr>\n   <td>\n    M1A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M2A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M3A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Parkwoods\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M4A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Victoria Village\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M5A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>\n    Regent Park, Harbourfront\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M6A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Lawrence Manor, Lawrence Heights\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M7A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>

In [6]:
# Put the parsed HTML into the dataframe

for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all("td")
    if (col != []): 
        PostalCode =col[0].text
        Borough = col[1].text
        Neighborhood = col[2].text
        df = df.append({"PostalCode":PostalCode, "Borough":Borough,'Neighborhood':Neighborhood}, ignore_index=True)

In [7]:
df.shape

(180, 3)

In [8]:
# Get rid of the /n string at the end of the strings

df['PostalCode'] = df['PostalCode'].map(lambda x: x.rstrip('\n'))
df['Borough'] = df['Borough'].map(lambda x: x.rstrip('\n'))
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.rstrip('\n'))

In [9]:
# Ignore cells where Borough is Not assigned

df = df[df['Borough'] != "Not assigned"]
df.reset_index(inplace=True)

In [10]:
df.shape

(103, 4)

In [11]:
# Getting the latitude and longitude with geocoder library

latitude=[]
longitude=[]
for code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M3A [43.75245000000007, -79.32990999999998]
M4A [43.73057000000006, -79.31305999999995]
M5A [43.65512000000007, -79.36263999999994]
M6A [43.72327000000007, -79.45041999999995]
M7A [43.66253000000006, -79.39187999999996]
M9A [43.662630000000036, -79.52830999999998]
M1B [43.811390000000074, -79.19661999999994]
M3B [43.74923000000007, -79.36185999999998]
M4B [43.70718000000005, -79.31191999999999]
M5B [43.65739000000008, -79.37803999999994]
M6B [43.70687000000004, -79.44811999999996]
M9B [43.65034000000003, -79.55361999999997]
M1C [43.78574000000003, -79.15874999999994]
M3C [43.72168000000005, -79.34351999999996]
M4C [43.68970000000007, -79.30681999999996]
M5C [43.65215000000006, -79.37586999999996]
M6C [43.69211000000007, -79.43035999999995]
M9C [43.64857000000006, -79.57824999999997]
M1E [43.765750000000025, -79.17469999999997]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M6E [43.68784000000005, -79.45045999999996]
M1G [43.76812000000007, -79.2

In [12]:
# Appending the latitude and longitude to the dataframe
df_geocoder = df.copy()
df_geocoder['Latitude'] = latitude
df_geocoder['Longitude'] = longitude

In [13]:
df_geocoder.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.75245,-79.32991
1,3,M4A,North York,Victoria Village,43.73057,-79.31306
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [14]:
# Alternatively, merge the dataframe with the the geospatial coordinates csv

coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()
df_merged = df.copy()
df_merged = pd.merge(left=df_merged, right=coords, left_on='PostalCode', right_on='Postal Code')
df_merged.head()

,index,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,2,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


Because of the unreliability of the geocoder, we decided to go forward with the version using the merged dataframes.

In [15]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto neighborhoods

In [16]:
# create map of New York using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

### Define foursquare credentials

In [17]:
CLIENT_ID = 'ZBH2Z0202NLDMAASF10XT1QS0CVEF2LCGF3Z4DGUXBKD4KCE' # your Foursquare ID
CLIENT_SECRET = '5KL0G531E0EF2NLPGY2MIX5ZOVFJXRQF4FX4UBJMX3PVERKB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
Limit = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZBH2Z0202NLDMAASF10XT1QS0CVEF2LCGF3Z4DGUXBKD4KCE
CLIENT_SECRET:5KL0G531E0EF2NLPGY2MIX5ZOVFJXRQF4FX4UBJMX3PVERKB


### Copy the necessary functions

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get the venues in 500 radius for each neighborhood

In [20]:
toronto_hood_venues = getNearbyVenues(names=df_merged['Neighborhood'],latitudes = df_merged['Latitude'],longitudes=df_merged['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [21]:
print('There are {} uniques categories.'.format(len(toronto_hood_venues['Venue Category'].unique())))

There are 270 uniques categories.


### Analyze the venues in each neighborhood

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_hood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_hood_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.0,0.0
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### Create a sorted dataframe which will display the 10 most common venues 

In [24]:
## Define the sorting function

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
# Create the dataframe

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] =  toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], 10)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Dance Studio,Pharmacy,Coffee Shop,Pub,Gym,American Restaurant,Falafel Restaurant,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Shopping Mall,Sandwich Place,Diner,Deli / Bodega,Restaurant,Intersection,Supermarket
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Thai Restaurant,Juice Bar,Fast Food Restaurant,Butcher,Café,Restaurant


### Clustering the neighborhoods (5 clusters)

In [26]:
# Make the dataframe ready for clustering

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels to the dataframe
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


# merge the original datarame with to sorted venues dataset 
df_merged = df_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,2.0,Bus Stop,Park,Hotel,Food & Drink Shop,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
1,3,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Donut Shop,Deli / Bodega
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,2.0,Coffee Shop,Bakery,Park,Café,Theater,Breakfast Spot,Pub,Bank,Hotel,French Restaurant
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763,2.0,Clothing Store,Accessories Store,Coffee Shop,Sporting Goods Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Event Space,Vietnamese Restaurant,Cosmetics Shop
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café


### Visualize the clusters

In [58]:
# Drop NA values from cluster labels so you can map it

df_merged = df_merged.dropna()

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters